# Vulnerability Assessment Pilot
This notebook demonstrates on-going development of climate adaptation vulnerability assessment (CAVA) support using climate data in the Analytics Engine. 

To execute a given 'cell' of this notebook, place the cursor in the cell and press the 'play' icon, or simply press shift+enter together. Some cells will take longer to run, and you will see a [$\ast$] to the left of the cell while AE is still working.

**Intended Application**: As a user, I want to **<span style="color:#FF0000">access climate projections data for my vulnerability assessment report</span>** by:
1. Retrieve data metrics required for planning needs

**Runtime**: With the default settings, this notebook takes approximately **less than 10 minutes** to run from start to finish. Modifications to selections may increase the runtime. <br>*This notebook is currently in progress, runtime will change as improvements and further analyses are added.*

### Step 0: Set-up

First, we'll import the Python library [climakitae](https://github.com/cal-adapt/climakitae), our AE toolkit for climate data analysis, along with this specific functions from that library that we'll use in this notebook, as well as any other necessary Python libraries to aid in analysis.

In [ ]:
from climakitae.explore.vulnerability import cava_data
from climakitae.explore.vulnerability_table import create_vul_table

### Step 1: Import locations
Now we'll read in point-based locations that we want to retrieve data for. For custom inputs, there are two options: (1) Input a single pair of latitude - longitude values; and (2) Import a csv file of locations that will each run. In the code below we show what each option looks like. 

Functionality to assess over a gridded area (region) is in the works as well!

In [ ]:
## select a single custom location
# your_lat = LAT
# your_lon = LON

To import your own custom locations, we recommend putting your csv file in the same folder as this notebook for ease:
1. Drag and drop a csv file into the file tree on the left hand side; or
2. Use the `upload` button (the "up arrow" symbol next to the large blue plus symbol above the file tree). 

<span style="color:#FF0000">**Formatting note**</span>: For the code cells below to work, there must be **2 columns labeled `lat` and `lon`**. Functionality to accept different labeling is forthcoming!

In the cell below, we read the csv file in. We extracted three random locations from the HadISD station list as an example here -- you'll want to replace with your own locations file!

In [ ]:
# Read in dummy locations from `stations_csv` file
from climakitae.core.paths import stations_csv_path
from climakitae.util.utils import read_csv_file
example_locs = read_csv_file(stations_csv_path, index_col=0)[['LAT_Y', 'LON_X']].rename(columns={'LAT_Y': 'lat', 'LON_X': 'lon'})

In [ ]:
# select a location from the location list
one_loc = example_locs.loc[example_locs.index == 0]
loc_lat = one_loc.lat.values[0]
loc_lon = one_loc.lon.values[0]
print(loc_lat, loc_lon)

### Step 2: Retrieve metric data

The `cava_data` funciontality is designed to provide flexibility over customizable metric calculation. There are 4 customizable metrics that can be built with this functionality:
1. Likely seasonal event occurence (e.g., "likely summer night low temperature")
2. 1-in-X temperature events (e.g., "1-in-10 year maximum temperature")
3. High/Extreme Heat Index events (e.g., "how many days per year does the Heat Index exceed 90°F")
4. 1-in-X precipitation events (e.g., "1-in-100 year, 24 hour precipitation *in progress*)

Below is a table outlining all avaialble arguments to the `cava_data` function. The "Required" flag notes whether the argument must be passed to start generating data. Input options for each argument is provided, as well as whether a setting is required for any of the required selections. We provide multiple examples of working with the `cava_data` function with multiple configurations.

| Argument | Options | Argument required for | Notes |
|----------|---------|-----------------------|-------|
|input_locations | Pass a location via csv. | All |*In progress direct input*|
|variable | "Air Temperature at 2m", "NOAA Heat Index"| All |*In progress "Precipitation (total)"* |
|approach | "time", "warming_level" | All | |
|downscaling_method | "Dynamical", "Statistical"| All | *In progress Statistical*|
|time_start_year | Numerical (min is 1981) | Required for **approach=time** | |
|time_end_year | Numerical (max is 2100) | Required for **approach=time**| |
|historical_data| "Historical Climate", "Historical Reconstruction" | Required for **approach=time**| *In progress Historical Reconstruction*|
|ssp_data | "[SSP2-4.5]", "[SSP3-7.0]", "[SSP5-8.5]" | Required for **approach=time** | Dynamical only has SSP3-7.0, Statisical has all 3|
|warming_level| "1.5", "2.0", "3.0" | Required for **approach=warming_level**| |
|metric_calc| "max", "min" | Required for 1-in-X temp, Heat Index, likely seasonal event | |
|heat_idx_threshold | Numerical | Required for Heat Index | |
|one_in_x | Numerical | Required for 1-in-X temp| |
|percentile | Numerical (0-100) | Required for likely seasonal event |   |
|season| "summer", "winter", "all" | Required for likely seasonal event| Default set to "all"|
|units | Temp/Heat Index: "degF", "degC", "K". Precip: "mm", "inches" | Optional | Default for temp/heat index is DegF. *In progress Precip*|
|wrf_bc| True, False | Optional| Option to select only the 4 bias-adjusted WRF models. Only applicable for **downscaling_method="Dynamical"**. Default set to True |
|export_method| "off-ramp", "calculate", "both" | Optional | Default set to "both" |
|file_format | "NetCDF", "csv" | Optional | Default set to "NetCDF" |
|separate_files | True, False | Optional | Option to export separate files if multiple points are passed. Default set to True|

#### Example: Likely seasonal event
Example scenario: I want to calculate "likely summer day high temperature for 2030-2050, where likely is the 75th percentile, in Celsius, using all available WRF data (bias-adjusted and non-bias-adjusted), and export only the calculated metric data". I would input:

In [ ]:
data = cava_data(
    ## Set-up
    example_locs.iloc[:1],
    time_start_year=2030, 
    time_end_year=2050,
    downscaling_method="Dynamical",  # default for now
    approach="time",  
    ssp_data=["SSP3-7.0"],
    wrf_bc=False, # return all WRF models
    
    ## Likely seaonal event specific arguments
    variable="Air Temperature at 2m", 
    metric_calc="max", # daily high temperature
    percentile=75, # likeliness percentile
    season="summer", # season
    units="degC", # change units
    
    ## Export
    export_method="calculate",  # export only calculated metric data
    file_format="NetCDF",
)

In [ ]:
import xarray as xr
test = xr.open_dataset('likely_seasonal_summer_max_2030 to 2050_3543424N_-11905524W.nc')
test

#### Example: 1-in-X event
Example scenario: I want to calculate "1-in-10 year maximum temperature, in Fahrenheit, for 2070-2090, using WRF data, and export both the raw and calculated metric data." I would input:

In [ ]:
data = cava_data(
    ## Set-up
    example_locs.iloc[:1],
    time_start_year=2070,
    time_end_year=2090,
    downscaling_method="Dynamical",  # default for now 
    approach="time",  
    ssp_data=["SSP3-7.0"],
    
    ## 1-in-X event specific arguments
    variable="Air Temperature at 2m",
    metric_calc="max", # daily maximum temperature
    one_in_x=10, # One-in-X
    units="degF", # change units
    
    ## Export
    export_method="both",
    file_format="NetCDF",
)

#### Example: Heat Index
Example scenario: I want to calculate "the number of days per year that the Heat Index exceeds 90°F between 2030-2060, and export only the raw data". I would input:

In [ ]:
data = cava_data(
    ## Set-up
    example_locs.iloc[:1],
    time_start_year=2030,
    time_end_year=2060,
    downscaling_method="Dynamical",  # default for now
    approach="time",  
    ssp_data=["SSP3-7.0"],
    
    ## Heat Index specific arguments
    variable="NOAA Heat Index", 
    metric_calc="max", # daily maximum
    heat_idx_threshold=90, # Heat Index Threshold
    units="degF", # change units
    
    ## Export
    export_method="off-ramp",
    file_format="NetCDF",
)

#### Example: Global Warming Level approach
Example scenario: I want to calculate "likely winter day high temperature with a 2°C warming level, where likely is the 60th percentile, in Celsius, using WRF data, and export both the raw and calculated metric data". I would input:

In [ ]:
data = cava_data(
    ## Set-up
    example_locs.iloc[:1],
    downscaling_method="Dynamical",  # default for now
    approach="warming_level",
    warming_level="2.0",
    
    ## Likely seaonal event specific arguments
    variable="Air Temperature at 2m", 
    metric_calc="max", # daily high temperature
    percentile=60, # likeliness percentile
    season="winter", # season
    units="degC", # change units
    
    ## Export
    export_method="both",
    file_format="NetCDF",
)

---

### Appendix: Table Generation Sample Code

Below, you'll find code that generates a table with different climate data metrics used in a CAVA Report. Feel free to run it and check it out! It is still very much in progress. **This will take 30+ min. to run.**

In [ ]:
%%time
percentile = 50
heat_idx_threshold = 80
one_in_x = 10 # currently, only can do `one_in_x` for one value at a time
df = create_vul_table(example_locs.iloc[[10]], percentile, heat_idx_threshold, one_in_x)
df